Assignments:

To close out this checkpoint, you're going to do three assignments. For the first assignment, you'll write up a short answer to a question. For the second two assignments, you'll do your work in Jupyter notebooks.

Please submit links to all your work below. This is not a graded checkpoint, but you should discuss your solutions with your mentor. Also, when you're done, compare your work to these example solutions.

1.Interpretation and significance:

Suppose that we would like to know how much families in the US are spending on recreation annually. We've estimated the following model:

𝑒𝑥𝑝𝑒𝑛𝑑𝑖𝑡𝑢𝑟𝑒 = 873 + 0.0012 𝑎𝑛𝑛𝑢𝑎𝑙_𝑖𝑛𝑐𝑜𝑚𝑒 + 0.00002 𝑎𝑛𝑛𝑢𝑎𝑙_𝑖𝑛𝑐𝑜𝑚𝑒^2 − 223.57 ℎ𝑎𝑣𝑒_𝑘𝑖𝑑𝑠

expenditure is the annual spending on recreation in US dollars, annual_income is the annual income in US dollars, and have_kids is a dummy variable indicating the families with children. Interpret the estimated coefficients. What additional statistics should be given in order to make sure that your interpretations make sense statistically. Write up your answer.

For interpretating this model we should have t-statistics or the associated p-values. If the coefficants are not statistically significant we should consider them as a zero and interpretant the model in different way. 

I assume that all coefficients are statistically significant and their p_value are less than 0.1.

So, constants in linear regression models correspond to the outcome value that happen when all the explanatory variables take a zero value. The constant terms make the residuals zero on average. The constant is also called bias. So, in this model, the bias term is 873. 

For dummies variable: Have_kid: On average, families with kids spending 223.57 US dollars less than families without kids on annual expenditure of recreation.

For continuse variable: As you can see, the relationship between annual_income and expenditure is quadratic. So, if annual_income increase $1000 the expenditure increase $1.2 plus 0.02 * annual_income or 2% of annual_income. 
????correct????

2.Weather model:

In this exercise, you'll work with the historical temperature data from the previous checkpoint. To complete this assignment, submit a link to a Jupyter notebook containing your solutions to the following tasks:

- First, load the dataset from the weatherinszeged table from Thinkful's database.

In [1]:
# Import some libraries and packages:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn import linear_model
import statsmodels.api as sm
import statsmodels.formula.api as smf
from scipy.stats import bartlett
from scipy.stats import levene
from statsmodels.tsa.stattools import acf
from scipy.stats import jarque_bera
from scipy.stats import normaltest
from sqlalchemy import create_engine
import warnings
warnings.filterwarnings(action="ignore")
%matplotlib inline
pd.options.display.float_format = '{:.2f}'.format

In [2]:
# Connect to dataset:
postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'weatherinszeged'
engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format( postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))
df = pd.read_sql_query('select * from weatherinszeged', con = engine)
engine.dispose()

In [3]:
# Look at the data set:
df.head()

,date,summary,preciptype,temperature,apparenttemperature,humidity,windspeed,windbearing,visibility,loudcover,pressure,dailysummary
0,2006-03-31 22:00:00+00:00,Partly Cloudy,rain,9.47,7.39,0.89,14.12,251.00,15.83,0.00,1015.13,Partly cloudy throughout the day.
1,2006-03-31 23:00:00+00:00,Partly Cloudy,rain,9.36,7.23,0.86,14.26,259.00,15.83,0.00,1015.63,Partly cloudy throughout the day.
2,2006-04-01 00:00:00+00:00,Mostly Cloudy,rain,9.38,9.38,0.89,3.93,204.00,14.96,0.00,1015.94,Partly cloudy throughout the day.
3,2006-04-01 01:00:00+00:00,Partly Cloudy,rain,8.29,5.94,0.83,14.10,269.00,15.83,0.00,1016.41,Partly cloudy throughout the day.
4,2006-04-01 02:00:00+00:00,Mostly Cloudy,rain,8.76,6.98,0.83,11.04,259.00,15.83,0.00,1016.51,Partly cloudy throughout the day.


- Build a linear regression model where your target variable is the difference between the apparenttemperature and the temperature. As explanatory variables, use humidity and windspeed.
- Now, estimate your model using OLS. Are the estimated coefficients statistically significant? 
- Are the signs of the estimated coefficients in line with your previous expectations? 
- Interpret the estimated coefficients.
- What are the relations between the target and the explanatory variables?

In [4]:
# Build a regression model:
# Y is the target variable:
Y = (df['apparenttemperature']-df['temperature'])
# X is the feature set which includes humidity, windspeed:
X = df[['humidity', 'windspeed']]
# We need to manually add a constant in statsmodels' sm
X = sm.add_constant(X)

results = sm.OLS(Y, X).fit()

results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.288
Model:                            OLS   Adj. R-squared:                  0.288
Method:                 Least Squares   F-statistic:                 1.949e+04
Date:                Sun, 08 Dec 2019   Prob (F-statistic):               0.00
Time:                        22:34:05   Log-Likelihood:            -1.7046e+05
No. Observations:               96453   AIC:                         3.409e+05
Df Residuals:                   96450   BIC:                         3.409e+05
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          2.4381      0.021    115.948      0.000       2.397       2.479
humidity      -3.0292      0.024   -126.479      0.000      -3.076      -2.982
windspeed     -0.1193      0.001   -176.164      0.000      -0.121      -0.118
==============================================================================
Omnibus:                     3935.747   Durbin-Watson:                   0.267
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             4613.311
Skew:                          -0.478   Prob(JB):                         0.00
Kurtosis:                       3.484   Cond. No.                         88.1
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

- the linear regression model: 

Y = (df['apparenttemperature']-df['temperature']) = 2.44 - 3.03 humidity - 0.12 windspeed ????correct????

- As you can see p_values for coefficients are less than 0.1 or 0.05. So, the estimated coefficients statistically significant. 
- We expect the difference between the apparenttemperature and the temperature increase by increasing humidity and decrease by increasing windspeed. So, the sign of the estimated coefficient for humidity is not as our expected and it is negative that means by increasing humidity the diferences will decreased. But, the sign of the estimated coefficient for windspeed is as our expected and they have negatively related which means by increasing windspeed the differences will decreased.
- By increasing one point of humidity the difference between the apparenttemperature and the temperature will decreased by 3.03 point and by increasing one point of windspeed diffrences will decreased by 0.12 point.
- As you can see the relations between the target and the explanatory variables is negatively wich means by increasing explanatory variables the differences will decreased.?????correct????

Next, include the interaction of humidity and windspeed to the model above and estimate the model using OLS. 
- Are the coefficients statistically significant? 
- Did the signs of the estimated coefficients for humidity and windspeed change? 
- Interpret the estimated coefficients.

In [5]:
# Y is the target variable
Y = (df['apparenttemperature']-df['temperature'])

# This is the interaction between humidity and windspeed:
df["humidity_windspeed"] = df.humidity * df.windspeed

# X is the feature set
X = df[['humidity', 'windspeed', 'humidity_windspeed']]

# We add a constant to the model as it's a best practice
# to do so every time!
X = sm.add_constant(X)

# We fit an OLS model using statsmodels
results = sm.OLS(Y, X).fit()

# We print the summary results
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.341
Model:                            OLS   Adj. R-squared:                  0.341
Method:                 Least Squares   F-statistic:                 1.666e+04
Date:                Sun, 08 Dec 2019   Prob (F-statistic):               0.00
Time:                        22:34:05   Log-Likelihood:            -1.6669e+05
No. Observations:               96453   AIC:                         3.334e+05
Df Residuals:                   96449   BIC:                         3.334e+05
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
const                  0.0839      0

- the linear regression model: 

Y = (df['apparenttemperature']-df['temperature']) = 0.08 + 0.18 humidity + 0.09 windspeed - 0.30 humidity_windspeed ????correct???

- As you can see, the estimated coefficients statistically significant, because p_values for coefficients are less than 0.1 or 0.05. 
- Yes, by including the interaction of humidity and windspeed to the model, the sign of both explanatory variables are changed from negative to positive. 
- By increasing one point of humidity the difference between the apparenttemperature and the temperature will increased by 0.18 point and by increasing one point of windspeed, diffrences will increased by 0.09 point. About interaction variable, by increasing one point of humidity, target will increased 0.18-(0.30*windspeed) and by increasing one point in windspeed target will increased 0.09-(0.30*humidity). ???? correct?????

3. House prices model:

In this exercise, you'll interpret your house prices model. To complete this assignment, submit a link to a Jupyter notebook containing your solutions to the following tasks:
- Load the houseprices data from Thinkful's database.

In [6]:
# Connect to the data base:
postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'houseprices'
engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format( postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))
df = pd.read_sql_query('select * from houseprices', con=engine)
engine.dispose()

In [7]:
# Look at the dataset:
df.head()

,id,mssubclass,mszoning,lotfrontage,lotarea,street,alley,lotshape,landcontour,utilities,...,poolarea,poolqc,fence,miscfeature,miscval,mosold,yrsold,saletype,salecondition,saleprice
0,1,60,RL,65.00,8450,Pave,None,Reg,Lvl,AllPub,...,0,None,None,None,0,2,2008,WD,Normal,208500
1,2,20,RL,80.00,9600,Pave,None,Reg,Lvl,AllPub,...,0,None,None,None,0,5,2007,WD,Normal,181500
2,3,60,RL,68.00,11250,Pave,None,IR1,Lvl,AllPub,...,0,None,None,None,0,9,2008,WD,Normal,223500
3,4,70,RL,60.00,9550,Pave,None,IR1,Lvl,AllPub,...,0,None,None,None,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.00,14260,Pave,None,IR1,Lvl,AllPub,...,0,None,None,None,0,12,2008,WD,Normal,250000


- Run your house prices model again and interpret the results. Which features are statistically significant, and which are not?

In [8]:
# I convert two categorical variables to numbers and concat to dataset:
df = pd.concat([df,pd.get_dummies(df.mszoning, prefix="mszoning", drop_first=True)], axis=1)
df = pd.concat([df,pd.get_dummies(df.utilities, prefix="utilities", drop_first=True)], axis=1)
dummy_column_names = list(pd.get_dummies(df.mszoning, prefix="mszoning", drop_first=True).columns) + list(pd.get_dummies(df.utilities, prefix="utilities", drop_first=True).columns)


In [9]:
# Now we build a simple linear regression model and estimate it using OLS:

# Y is the target variable:
Y = df['saleprice']
# X is the feature set which includes: threessnporch, mosold and dummy columns:
X = df[['threessnporch','mosold'] + dummy_column_names]

# We create a LinearRegression model with statsmodels' sm linear_model module.

# We need to manually add a constant in statsmodels' sm
X = sm.add_constant(X)

# fit method estimates the coefficients using OLS
results = sm.OLS(Y,  X).fit()

# Inspect the results.
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              saleprice   R-squared:                       0.111
Model:                            OLS   Adj. R-squared:                  0.107
Method:                 Least Squares   F-statistic:                     25.98
Date:                Sun, 08 Dec 2019   Prob (F-statistic):           1.13e-33
Time:                        22:34:09   Log-Likelihood:                -18458.
No. Observations:                1460   AIC:                         3.693e+04
Df Residuals:                    1452   BIC:                         3.697e+04
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
const             6.348e+04   2.44e+04      2.602      0.009    1.56e+04    1.11e+05
threessnporch       84.8014     67.158      1.263      0.207     -46.936     216.539
mosold            1435.3758    729.741      1.967      0.049       3.916    2866.835
mszoning_FV       1.415e+05   2.55e+04      5.545      0.000    9.15e+04    1.92e+05
mszoning_RH       5.732e+04   3.03e+04      1.894      0.058   -2045.410    1.17e+05
mszoning_RL       1.181e+05   2.39e+04      4.950      0.000    7.13e+04    1.65e+05
mszoning_RM       5.387e+04   2.43e+04      2.217      0.027    6195.085    1.02e+05
utilities_NoSeWa -4.556e+04   7.52e+04     -0.606      0.545   -1.93e+05    1.02e+05
==============================================================================
Omnibus:                      692.221   Durbin-Watson:                   2.006
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             5041.452
Skew:                           2.081   Prob(JB):                         0.00
Kurtosis:                      11.096   Cond. No.                     1.13e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.13e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

Now, modeling the saleprice houses with linear regression is:

saleprice:y = 63480 + 84.8014 threessnporch + 1435.3758 mosold + 370790 mszoning - 45560 utilities ???? correct???

- All explanatory variables are  statistically significant except threessnporch and utilities_NoSeWa which are statistically insignificant, because their p_values are greater than 0.05 or 0.1. So their coefficients are statistically zero.
- By increasing one point of mosold saleprice will increased 1435.38. Based on coefficents of mszoning, average saleprice is higher in mszoning_FV (141500), mszoning_RL (118100), mszoning_RH (57320) and mszoning_RM (53870) respectively.??? correct????

- Now, exclude the insignificant features from your model. Did anything change?

In [10]:
# Now we build a simple linear regression model and estimate it using OLS:

# Y is the target variable:
Y = df['saleprice']
# X is the feature set which includes: mosold, mszoning_FV, mszoning_RL, mszoning_RH, mszoning_RM:
X = df[['mosold', 'mszoning_FV', 'mszoning_RL', 'mszoning_RH', 'mszoning_RM']]

# We create a LinearRegression model with statsmodels' sm linear_model module.

# We need to manually add a constant in statsmodels' sm
X = sm.add_constant(X)

# fit method estimates the coefficients using OLS
results = sm.OLS(Y,  X).fit()

# Inspect the results.
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              saleprice   R-squared:                       0.110
Model:                            OLS   Adj. R-squared:                  0.107
Method:                 Least Squares   F-statistic:                     35.98
Date:                Sun, 08 Dec 2019   Prob (F-statistic):           8.10e-35
Time:                        22:34:09   Log-Likelihood:                -18459.
No. Observations:                1460   AIC:                         3.693e+04
Df Residuals:                    1454   BIC:                         3.696e+04
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const        6.309e+04   2.44e+04      2.586      0.010    1.52e+04    1.11e+05
mosold       1485.7192    728.437      2.040      0.042      56.820    2914.618
mszoning_FV  1.416e+05   2.55e+04      5.548      0.000    9.15e+04    1.92e+05
mszoning_RL  1.185e+05   2.39e+04      4.967      0.000    7.17e+04    1.65e+05
mszoning_RH  5.733e+04   3.03e+04      1.894      0.058   -2034.552    1.17e+05
mszoning_RM    5.4e+04   2.43e+04      2.222      0.026    6330.576    1.02e+05
==============================================================================
Omnibus:                      691.059   Durbin-Watson:                   2.004
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             5014.888
Skew:                           2.078   Prob(JB):                         0.00
Kurtosis:                      11.072   Cond. No.                         192.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

saleprice:y = 63090 + 1485.72 mosold + 141600 mszoning_FV + 118500 mszoning_RL + 57330 mszoning_RH + 54000 mszoning_RM
- As you can see,the continuse variable and dummies variables have changed a little.???? correct??

- Interpret the statistically significant coefficients by quantifying their relations with the house prices. Which features have a more prominent effect on house prices?

- As you can see, all coefficients are statistically significant because all p_values are less than 0.05 or 0.1.
- By increasing one point of mosold, saleprice will increased 1485.72. Based on coefficents of mszoning, average saleprice is higher in mszoning_FV (141600) and mszoning_RL (118500), mszoning_RH (57330) and mszoning_RM (54000) respectively.
- The largest coefficient is for mszoning_FV which has a more prominent effect on house prices.?????correct???

- Do the results sound reasonable to you? If not, try to explain the potential reasons.

I think house saleprice depends on every thing such as location, overall quality, size, month of sale, zoning, etc. I investigated some categorical values, also, some continuse variables with lowest relationship with saleprice, so it makes sense the result didnt seem reasonable much. ??????????correct??????????